In [1]:
import json
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
stopwords_list = stopwords.words("russian")

### Детально рассмотрим каждый шаг предобработки на первом документе корпуса

In [3]:
with open('edited_corpus.json', 'r') as f:
    corpus = json.load(f)
print(corpus[0])

Миф об обязательных вибрациях при выходе из тела  ПРАКТИКА    Вибрации - это признак скорого выхода из тела или, если угодно, выхода в осознанный сон. Учёные связывают это с процессом возникновения сновидения. В этом состоянии тело начинает входить в паралич, чтобы не повторять движения из сна     Вибрации похожи на дрожь от электрического тока, только без боли, либо на дрожь от холода. Если вы испытываете вибрации, значит, из этого состояние очень легко выйти из тела!    Вибрации испытывал каждый, кто хоть раз говорил: «Меня аж передёрнуло от этого»  Только такие вибрации длятся секунду, а вибрации при выходе из тела чуть дольше.    Для того, чтобы почувствовать это состоянии прямо сейчас сделайте следующее: закройте глаза и представьте точку на уровне глаз на расстоянии 40 см и медленно её отдаляйте от тела. Чувствуете вибрации? У меня они наступают даже без расслабления.    Но надо понимать, что вибрации не всегда наступают при выходе из тела. Хотя они и случаются часто. Пока непоня

### Разобьем текст на токены с помощью RegexpTokenizer библиотеки nltk (обычный word_tokenize оставляет много пунктуации). В полученном списке токенами считаются также буквы т и д из т.д. и числительные. Избавимся от них с помощью условия на соответствие isalpha() и длины токена > 1, попутно приведя токены к нижнему регистру

In [4]:
tokenized_corpus = corpus[23]
re_tokenizer = RegexpTokenizer(r'[А-яA-z-ё]+')
tokens = re_tokenizer.tokenize(tokenized_corpus)
print(tokens[:50])

['Мешает', 'ли', 'сон', 'с', 'животными', 'практике', 'осознанных', 'снов', 'женщины', 'оценили', 'сон', 'с', 'домашними', 'питомцами', 'Любимое', 'животное', 'которое', 'спит', 'рядом', 'может', 'как', 'помочь', 'увидеть', 'осознанный', 'сон', 'так', 'и', 'навредить', 'Например', 'собака', 'может', 'залаять', 'кошка', 'замяукать', 'и', 'просто', 'вас', 'разбудить', 'из', 'осознанного', 'сна', 'А', 'ещё', 'животное', 'может', 'шуметь', 'шуршать', 'царапаться', 'или', 'толкаться']


In [5]:
tokens = [token.lower() for token in tokens if len(token) > 1 and token.isalpha()]
print(tokens[:50])

['мешает', 'ли', 'сон', 'животными', 'практике', 'осознанных', 'снов', 'женщины', 'оценили', 'сон', 'домашними', 'питомцами', 'любимое', 'животное', 'которое', 'спит', 'рядом', 'может', 'как', 'помочь', 'увидеть', 'осознанный', 'сон', 'так', 'навредить', 'например', 'собака', 'может', 'залаять', 'кошка', 'замяукать', 'просто', 'вас', 'разбудить', 'из', 'осознанного', 'сна', 'ещё', 'животное', 'может', 'шуметь', 'шуршать', 'царапаться', 'или', 'толкаться', 'всё', 'это', 'может', 'вас', 'сбить']


### Наконец, избавимся от стоп-слов (список импортирован из nltk).\
Добавим пару своих слов.

In [6]:
stopwords_list.extend(['из-за', 'аж'])
wout_sw = [word for word in tokens if word not in stopwords_list]
print(wout_sw[:50])

['мешает', 'сон', 'животными', 'практике', 'осознанных', 'снов', 'женщины', 'оценили', 'сон', 'домашними', 'питомцами', 'любимое', 'животное', 'которое', 'спит', 'рядом', 'помочь', 'увидеть', 'осознанный', 'сон', 'навредить', 'например', 'собака', 'залаять', 'кошка', 'замяукать', 'просто', 'разбудить', 'осознанного', 'сна', 'ещё', 'животное', 'шуметь', 'шуршать', 'царапаться', 'толкаться', 'всё', 'это', 'сбить', 'техники', 'выхода', 'тела', 'вернуть', 'осознанного', 'сна', 'реальность', 'стороны', 'домашние', 'животные', 'которые']


### Создадим словарь с обработанными документами

In [12]:
def preprocess(doc: str):
    re_tokenizer = RegexpTokenizer(r'[А-яA-z-ё]+')
    tokens = re_tokenizer.tokenize(doc)
    tokens = [token.lower() for token in tokens if len(token) > 1 and token.isalpha()]
    tokenized_doc = [word for word in tokens if word not in stopwords_list]
    return tokenized_doc

In [13]:
dict_tokenized_corpus = {}
for i, doc in enumerate(corpus):
    dict_tokenized_corpus[i] = preprocess(doc)

In [14]:
with open('dict_tokenized_corpus.json', 'w') as f:
    json.dump(dict_tokenized_corpus, f, ensure_ascii=False)

### Реализация функции для нахождения вхождений слова и слов с опечатками

In [10]:
from nltk.metrics import edit_distance

def find_entries(input_word: str, corpus=dict_tokenized_corpus):
    ids = []
    typos = []
    for key, doc in corpus.items():
        if input_word in doc:
            ids.append(key)
    # if no entries found
    if not ids:
        for key, doc in corpus.items():
            for word in doc:
                if edit_distance(input_word, word) <= 1:
                    typos.append(word)
                    ids.append(key)
    print(f'Для "{input_word}"')
    print(f'Похожие: {typos}')
    return ids

### Примеры работы функции:

In [11]:
print(find_entries('специалист'))
print(find_entries('придти'))
print(find_entries('коллапс'))

Для "специалист"
Похожие: []
[94, 95, 112]
Для "придти"
Похожие: ['прийти', 'прийти', 'прийти', 'прийти']
[125, 139, 247, 260]
Для "коллапс"
Похожие: []
[]
